<a href="https://colab.research.google.com/github/Hemant-ML/Pytorch-Master/blob/main/_downloads/ea0953da74a725b00248ac3fceada0f6/buildmodel_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

Get Device for Training
-----------------------
We want to be able to train our model on a hardware accelerator like the GPU, 
if it is available. Let's check to see if 
`torch.cuda.is available`, else we 
continue to use the CPU. 



In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using {device} device')

Using cuda device


Define the Class
-------------------------
We define our neural network by subclassing ``nn.Module``, and 
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method. 



In [5]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork,self).__init__()
    self.flatten=nn.Flatten()
    self.linear_Relu = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,10),
        nn.ReLU()
    )
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_Relu(x)
    return logits




We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print 
its structure.



In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_Relu): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
    (3): ReLU()
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some `background operations`_. 
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) 
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


In [9]:
X.size()

torch.Size([1, 28, 28])

--------------




Model Layers
-------------------------

Lets break down the layers in the FashionMNIST model. To illustrate it, we 
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as 
we pass it through the network. 



In [14]:
input_image = torch.rand(3,28,28)
input_image.view(3,-1).size()

torch.Size([3, 784])

``nn.Flatten``

We initialize the `nn.Flatten `_ 
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).



In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


``nn.Linear``

The `linear layer` 
is a module that applies a linear transformation on the input using its stored weights and biases.




In [15]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


``nn.ReLU``
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use `nn.ReLU` between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

``nn.Sequential``

`nn.Sequential` is an ordered 
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax

The last linear layer of the neural network returns `logits` - raw values in [-infty, infty] - which are passed to the
`nn.Softmax >`_ module. The logits are scaled to values 
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along 
which the values must sum to 1. 



In [17]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
-------------------------
Many layers inside a neural network are *parameterized*, i.e. have associated weights 
and biases that are optimized during training. Subclassing ``nn.Module`` automatically 
tracks all fields defined inside your model object, and makes all parameters 
accessible using your model's ``parameters()`` or ``named_parameters()`` methods. 

In this example, we iterate over each parameter, and print its size and a preview of its values. 




In [18]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_Relu): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
    (3): ReLU()
  )
) 


Layer: linear_Relu.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0335,  0.0250,  0.0209,  ...,  0.0153,  0.0081,  0.0167],
        [ 0.0095, -0.0353,  0.0291,  ...,  0.0094,  0.0332, -0.0098]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_Relu.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0289, -0.0017], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_Relu.2.weight | Size: torch.Size([10, 512]) | Values : tensor([[ 0.0230, -0.0332, -0.0031,  ..., -0.0134,  0.0198,  0.0206],
        [-0.0008,  0.0233, -0.0377,  ...,  0.0204, -0.0129, -0.0059]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_Relu.2.bias | Size: torch.Size([10]) | Values : tensor([ 

--------------


